<a href="https://colab.research.google.com/github/teang1995/satellite_image_classification/blob/master/simple_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**import Libraries**

In [0]:
import os
import shutil
from PIL import Image
import torch
from torch.utils.data import Dataset,DataLoader
import numpy as np
from torchvision import transforms as T
import torchvision
import cv2
import sys
from google.colab import drive
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

**check cuda**

In [0]:
torch.cuda.is_available()

True

**drive mount**

In [0]:
drive.mount('/content/drive')

**Dataset Class**

In [0]:
class NWPU_RESISC45(Dataset):
  '''
  1. Satellite image dataset
  2. 45 classes
  '''
  #initiallize my data , download
  def __init__(self, root, data_list_file, phase = 'train',transform = None):
    self.phase = phase
    #요 부분 코드 수정 필요해보임!
    #현재 data_list_file이 txt형식으로 돼있어보인다.
    #나는 excel혹은 csv파일 꼴로 만들고 싶으니 그에 맞게 수정이 필요함.
    with open(os.path.join(data_list_file), 'r') as fd:
            imgs = fd.readlines()

    imgs = [os.path.join(root, img[:-1]) for img in imgs]
    self.imgs = np.random.permutation(imgs)
    normalize = T.Normalize(mean=[0.5], std=[0.5])
    self.transform = transform
  #파일명은 현재 classname_000.jpg꼴로 돼있음.
  #1.파일명에서 substring으로 label을 따올지
  #2.csv파일을 작성시에 label을 따로 저장해둘지.
  #를 결정한 후 getitem함수 수정 필요함.
  def __getitem__(self,index):
    sample = self.imgs[index]
    imp_path = sample
    data = Image.open(img_path)
    #data = data.convert('L') #binary chaneel
    data = self.transform(data)
    label = "label" #수정 필요
    return data.float(), label

  def __len__(self):
    return self.len

**transform and Dataloader**

In [0]:
transform = T.Compose([transforms.ToTensor()])
batch_size = 64
classes = os.listdir("/content/drive/My Drive/NWPU-RESISC45")

trainset = NWPU_RESISC45(root = "'''뭘 넣어야 하지'''",split = 'train',transform = transform)
trainloader = DataLoader(trainset,batch_size = batch_size)

validationset = NWPU_RESISC45(root = "'''뭘 넣어야 하지'''",split = 'validation',transform = transform)
validloader = DataLoader(validationset,batch_size = batch_size)
testset = NWPU_RESISC45(root = "'''뭘 넣어야 하지'''",split = 'test',transform = transform)
testloader = DataLoader(testset,batch_size = batch_size)

**VGGNET**

In [0]:
class Net(nn.Module) :
  def __init__(self):
    super(Net,self).__init__()
    self.conv = nn.Sequential(
        #3 224 224
        nn.Conv2d(3,64,3,padding = 1),nn.ReLU(),
        nn.Conv2d(64,64,3,padding = 1),nn.ReLU(),
        nn.MaxPool2d(2,2),
        #64 112 112
        nn.Conv2d(64, 128, 3, padding=1),nn.LeakyReLU(0.2),
        nn.Conv2d(128, 128, 3, padding=1),nn.LeakyReLU(0.2),
        nn.MaxPool2d(2, 2),
        #128 56 56
        nn.Conv2d(128, 256, 3, padding=1),nn.LeakyReLU(0.2),
        nn.Conv2d(256, 256, 3, padding=1),nn.LeakyReLU(0.2),
        nn.Conv2d(256, 256, 3, padding=1),nn.LeakyReLU(0.2),
        nn.MaxPool2d(2, 2),
        #256 28 28
        nn.Conv2d(256, 512, 3, padding=1),nn.LeakyReLU(0.2),
        nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
        nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
        nn.MaxPool2d(2, 2),
        #512 14 14
        nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
        nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
        nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
        nn.MaxPool2d(2, 2)
    )
        #512 7 7
    self.fc1 = nn.Linear(512 * 7 * 7,4096)
    self.fc2 = nn.Linear(4096,4096)
    self.fc3 = nn.Linear(4096,45)

  def forward(self,x):
    features = self.conv(x)
    x = x.view(features.size(0),-1)
    x = fc1(x)
    x = fc2(x)
    x = fc3(x)
    x = self.softmax(x)
    return x
#cuda사용 가능하면 gpu사용, 아니면 cpu로 연산
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = Net()
net = net.to(device)


**optimizer and loss function**

In [0]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(net.parameters(),lr = 0.00001)

**Test and Validation**

In [0]:
imgs = "/content/drive/My Drive/dataset/Test"


['b', 'c']
